# PSET 6

LINK: https://github.com/benjaminzaidel/MSE228New/blob/master/PSET6_228.ipynb

# Question 1

## a)

What was the estimate of the causal effect of the treatment and what is the 95% confidence interval? The second OLS model predicts the outcome from just the controls. Which control factors were statistically significant at the 95% level? The third OLS model predicts treatment from the controls. Which control factors were statistically significant at the 95% level? Subsequently, we estimate the same treatment effect using double lasso, and control for third degree polynomials. What was the estimate and 95% percent confidence interval from the double lasso, and compare your results with that from the OLS model. 


<span style = "color: red"> 


Estimate of causal effect of treatment based on colab: **0.0973**

95% confidence interval: **[0.051,0.144]**

Statistically significant control factors in second model: **femaleR, ageR**

Statistically significant control factors in THIRD model: **herderR**

Causal effect estimate for double lasso: **0.1003**

95% confidence interval: **[0.052,0.148]**

We see very similar results between the OLS model and the double lasso model (almost the exact same numbers). This suggests that OLS was already controlling for key confounders, meaning no major omitted variable bias. This also indicates that high-dimensional interactions or nonlinearities didn’t significantly impact the treatment effect, and the results are robust across methods. Since the confidence intervals are also similar, both methods provide consistent and stable estimates, increasing confidence in the causal effect.


</span>

## b)

The next part of the notebook analyzes the bias from unobserved confounding. The analyst needs to provide a partial R-square for both the outcome and treatment that can be further explained by some unobserved confounding. Replace the current R-square in the notebook with the R-squares from the OLS models. In particular, the R-square for the outcome Y should be set to the R-square from the OLS predicting the outcome just from the controls. Similarly, the R-square for the treatment should be set to the R-square from the OLS predicting the treatment just from the controls.This way we are positing that the unobserved confounders are as strong of a control as the controls that we already have. What is the bias that such an unobserved confounder can cause, and is it sufficient to overturn the treatment finding from the double lasso, i.e. if we subtract this bias from the lower end of the CI, will we get a non-positive treatment effect? Then we plot the level set of the bias, as we vary the R-square of the treatment and the R-square of the outcome. Looking at the plot, if the R-square for the treatment that the unobserved confounder further explains was 0.1, approximately how much should the R-square of the outcome be, so that the unobserved confounder can lead to the same amount of bias?

In [ ]:
# Main estimate
beta = dml_model.params[1]

# Hypothetical values of partial R2s
R2_YC = 0.135
R2_DC = 0.018

# Elements of the bias equation
kappa = (R2_YC * R2_DC) / (1 - R2_DC)
variance_ratio = np.mean(dml_model.resid**2) / np.mean(resD**2)

# Compute square bias
BiasSq = kappa * variance_ratio

# Compute absolute value of the bias
print("absolute value of the bias:", np.sqrt(BiasSq))

# Plotting
gridR2_DC = np.arange(0, 0.301, 0.001)
gridR2_YC = kappa * (1 - gridR2_DC) / gridR2_DC
gridR2_YC = np.where(gridR2_YC > 1, 1, gridR2_YC)

#Extra print line
print("Curve value at x = 0.1:", gridR2_YC[np.where(gridR2_DC == 0.1)][0])

plt.plot(gridR2_DC, gridR2_YC, color='red')
plt.xlabel('Partial R2 of Treatment with Confounder')
plt.ylabel('Partial R2 of Outcome with Confounder')
plt.title(f'Combo of R2 such that |Bias| < {np.round(np.sqrt(BiasSq), decimals=4)}')
plt.show()

<span style = "color: red"> 

Absolute value of the bias: **0.032444448889241514** 

Curve value at x = 0.1: 0.022270875763747457

Even after we subtract this value from the lower bound of confidence interval [0.052,0.148], we get [0.01955555111, 0.148]. As this is >0, there is no need to overturn the treatment finding.

Finally, given our point (0.1, 0.022270875763747457), we can conclude that if the unobserved confounder explained 10% of the variation in the treatment, it would only need to explain ~2.23% of the variation in the outcome to produce the same level of bias.

</span>

![alt text](outputpartb.png "Test")

## c)

The rest of the code packages everything into a set of functions that captures these functionalities, but also captures the uncertainty in the calculation of the bias. For instance, dml_sensitivity_bounds produces upper and lower bounds as a function of the partial R-square of the outcome and the partial R-square of the treatment. Call this function to get sensitivity bound for the same positive R-squared values as you did in the previous calculations, then call it again and incorporate the uncertainty from the bias calculation with 95% confidence. Does incorporating this extra uncertainty overturn the results? Keeping the R-square of the outcome the same, what is the smallest R-square of the treatment needed to overturn the treatment effect estimation?


In [ ]:
no_uncertainty = dml_sensitivity_bounds((resY, resD), R2_YC, R2_DC)
print(no_uncertainty)

uncertainty = dml_sensitivity_bounds((resY, resD), R2_YC, R2_DC, alpha=0.05)
print(uncertainty)



With no uncertainty: (0.0678919992440549, 0.13278089702253792)

With uncertainty (alpha = 0.05): (0.023034680259483034, 0.1775960026935742)

In [ ]:
no_uncertainty = dml_sensitivity_bounds((resY, resD), R2_YC, R2_DC)
print(no_uncertainty)

uncertainty = dml_sensitivity_bounds((resY, resD), R2_YC, R2_DC, alpha=0.05)
print(uncertainty)

R2_DC_max = 0.018
while dml_sensitivity_bounds((resY, resD), R2_YC, R2_DC_max, alpha=0.05)[0]>0:
    R2_DC_max += 0.0001
print(R2_DC_max)
print(dml_sensitivity_bounds((resY, resD), R2_YC, R2_DC_max, alpha=0.05))


(0.0678919992440549, 0.13278089702253792)
(0.023034680259483034, 0.1775960026935742)
0.05090000000000048
(-1.701343652082965e-05, 0.20064798164717013)

Even with uncertainty, lower bound remains positive, showing that uncertainty does not overturn the results. 

Highest R2_DC_max (R^2 of treatment) you can set before results get overturned is ~0.0509.

# Question 2

## a) 

Repeat the analysis in that notebook for the low income (bottom 25% of the income distribution) and for the high income groups (top 25% of the income distribution). Report the tables with the results under the PLIV and the corresponding interactive IV (LATE) model assumptions. Comment on whether there is any heterogeneity on the effect of 401k with respect to income. Comment on whether the different ML methods provide consistent estimates with each other, for each of these income groups.

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LassoCV, LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import (GradientBoostingRegressor, GradientBoostingClassifier,
                              RandomForestRegressor, RandomForestClassifier)
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.base import BaseEstimator, clone, TransformerMixin
import scipy.stats
import warnings
warnings.simplefilter('ignore')
import os
os.environ["PYTHONWARNINGS"] = "ignore"
np.random.seed(1234)

# Load data and define groups
file = "https://raw.githubusercontent.com/CausalAIBook/MetricsMLNotebooks/main/data/401k.csv"
data_all = pd.read_csv(file)
data_bottom = data_all.loc[data_all['inc'] < data_all['inc'].quantile(0.25)].copy()
data_top = data_all.loc[data_all['inc'] > data_all['inc'].quantile(0.75)].copy()

# A simple transformer using a formula (returns a numpy array)
from formulaic import Formula
class FormulaTransformer(TransformerMixin, BaseEstimator):
    def __init__(self, formula, array=False):
        self.formula = formula
        self.array = array
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        df = Formula(self.formula).get_model_matrix(X)
        return df.values if self.array else df

transformer = FormulaTransformer(
    "0 + poly(age, degree=6, raw=True) + poly(inc, degree=8, raw=True) "
    "+ poly(educ, degree=4, raw=True) + poly(fsize, degree=2, raw=True) "
    "+ male + marr + twoearn + db + pira + hown", array=True)

# Columns to drop from the dataset when forming controls X
drop_cols = ['e401', 'p401', 'a401', 'tw', 'tfa', 'net_tfa', 'tfa_he',
             'hval', 'hmort', 'hequity',
             'nifa', 'net_nifa', 'net_n401', 'ira',
             'dum91', 'icat', 'ecat', 'zhat',
             'i1', 'i2', 'i3', 'i4', 'i5', 'i6', 'i7',
             'a1', 'a2', 'a3', 'a4', 'a5']


class TransformedEstimator(BaseEstimator):
    def __init__(self, transformer, estimator):
        self.transformer = transformer
        self.estimator = estimator
    def fit(self, X, y):
        X_trans = self.transformer.fit_transform(X)
        self.estimator.fit(X_trans, y)
        return self
    def predict(self, X):
        X_trans = self.transformer.transform(X)
        return self.estimator.predict(X_trans)
    def predict_proba(self, X):
        X_trans = self.transformer.transform(X)
        return self.estimator.predict_proba(X_trans)


# Functions for PLIV Analysis

def dml(X, Z, D, y, modely, modeld, modelz, *, nfolds, classifier=False):
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=123)
    yhat = cross_val_predict(modely, X, y, cv=cv, n_jobs=-1)
    if classifier:
        Dhat = cross_val_predict(modeld, X, D, cv=cv, method='predict_proba', n_jobs=-1)[:, 1]
        Zhat = cross_val_predict(modelz, X, Z, cv=cv, method='predict_proba', n_jobs=-1)[:, 1]
    else:
        Dhat = cross_val_predict(modeld, X, D, cv=cv, n_jobs=-1)
        Zhat = cross_val_predict(modelz, X, Z, cv=cv, n_jobs=-1)
    resy = y - yhat
    resD = D - Dhat
    resZ = Z - Zhat
    point = np.mean(resy * resZ) / np.mean(resD * resZ)
    epsilon = resy - point * resD
    var = np.mean(epsilon**2 * resZ**2) / np.mean(resD * resZ)**2
    stderr = np.sqrt(var / X.shape[0])
    return point, stderr, yhat, Dhat, Zhat, resy, resD, resZ, epsilon

def summary(point, stderr, yhat, Dhat, Zhat, resy, resD, resZ, epsilon, X, Z, D, y, *, name):
    return pd.DataFrame({
        'estimate': point,
        'stderr': stderr,
        'lower': point - 1.96 * stderr,
        'upper': point + 1.96 * stderr,
        'rmse y': np.sqrt(np.mean(resy**2)),
        'rmse D': np.sqrt(np.mean(resD**2)),
        'rmse Z': np.sqrt(np.mean(resZ**2)),
        'accuracy D': np.mean(np.abs(resD) < 0.5),
        'accuracy Z': np.mean(np.abs(resZ) < 0.5)
    }, index=[name])

def robust_inference(point, stderr, yhat, Dhat, Zhat, resy, resD, resZ, epsilon, X, Z, D, y, *, grid, alpha=0.05):
    n = X.shape[0]
    thr = scipy.stats.chi2.ppf(1 - alpha, df=1)
    accept = []
    for theta in grid:
        moment = (resy - theta * resD) * resZ
        test = n * np.mean(moment)**2 / np.var(moment)
        if test <= thr:
            accept.append(theta)
    return accept

def run_pliv_analysis(data, desc, transformer):
    print(f"---------- PLIV Analysis for {desc} ----------")
    y = data['net_tfa'].values
    Z = data['e401'].values
    D = data['p401'].values
    X = data.drop(drop_cols, axis=1)
    
    results = []
    cv = KFold(n_splits=5, shuffle=True, random_state=123)
    
    # Method 1: Double Lasso
    lassoy = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv))
    lassod = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv))
    lassoz = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv))
    res = dml(X, Z, D, y, lassoy, lassod, lassoz, nfolds=3)
    results.append(summary(*res, X, Z, D, y, name='double lasso'))
    
    # Method 2: Lasso + Logistic Regression
    lgrd = make_pipeline(transformer, StandardScaler(), LogisticRegressionCV(cv=cv))
    lgrz = make_pipeline(transformer, StandardScaler(), LogisticRegressionCV(cv=cv))
    res = dml(X, Z, D, y, lassoy, lgrd, lgrz, nfolds=3, classifier=True)
    results.append(summary(*res, X, Z, D, y, name='lasso/logistic'))
    
    # Method 3: Random Forests
    rfy = make_pipeline(transformer, RandomForestRegressor(n_estimators=100, min_samples_leaf=10, ccp_alpha=0.001))
    rfd = make_pipeline(transformer, RandomForestClassifier(n_estimators=100, min_samples_leaf=10, ccp_alpha=0.001))
    rfz = make_pipeline(transformer, RandomForestClassifier(n_estimators=100, min_samples_leaf=10, ccp_alpha=0.001))
    res = dml(X, Z, D, y, rfy, rfd, rfz, nfolds=3, classifier=True)
    results.append(summary(*res, X, Z, D, y, name='random forest'))
    
    # Method 4: Decision Trees
    dtry = make_pipeline(transformer, DecisionTreeRegressor(min_samples_leaf=10, ccp_alpha=0.001))
    dtrd = make_pipeline(transformer, DecisionTreeClassifier(min_samples_leaf=10, ccp_alpha=0.001))
    dtrz = make_pipeline(transformer, DecisionTreeClassifier(min_samples_leaf=10, ccp_alpha=0.001))
    res = dml(X, Z, D, y, dtry, dtrd, dtrz, nfolds=3, classifier=True)
    results.append(summary(*res, X, Z, D, y, name='decision tree'))
    
    # Method 5: Boosted Trees
    gbfy = make_pipeline(transformer, GradientBoostingRegressor(max_depth=2, n_iter_no_change=5))
    gbfd = make_pipeline(transformer, GradientBoostingClassifier(max_depth=2, n_iter_no_change=5))
    gbfz = make_pipeline(transformer, GradientBoostingClassifier(max_depth=2, n_iter_no_change=5))
    res = dml(X, Z, D, y, gbfy, gbfd, gbfz, nfolds=3, classifier=True)
    results.append(summary(*res, X, Z, D, y, name='boosted forest'))
    
    # Method 6: AutoML via FLAML (try/except to bypass if errors occur)
    try:
        from flaml import AutoML
        X_trans = transformer.fit_transform(X)
        automl_reg = AutoML(time_budget=100, task='regression', early_stop=5, max_iter=100,
                            eval_method='cv', n_splits=3, metric='r2', verbose=0)
        automl_reg.fit(X_trans, y)
        best_reg = automl_reg.best_estimator
        automl_clf = AutoML(time_budget=100, task='classification', early_stop=5, max_iter=100,
                            eval_method='cv', n_splits=3, metric='r2', verbose=0)
        automl_clf.fit(X_trans, D)
        best_d = automl_clf.best_estimator
        automl_clf.fit(X_trans, Z)
        best_z = automl_clf.best_estimator
        modely_auto = TransformedEstimator(transformer, clone(best_reg))
        modeld_auto = TransformedEstimator(transformer, clone(best_d))
        modelz_auto = TransformedEstimator(transformer, clone(best_z))
        res_auto = dml(X, Z, D, y, modely_auto, modeld_auto, modelz_auto, nfolds=3, classifier=True)
        results.append(summary(*res_auto, X, Z, D, y, name='automl (semi-cfit)'))
    except Exception as e:
        print("FLAML AutoML failed, skipping AutoML method. Error:", e)
    
    table_pliv = pd.concat(results)
    print(table_pliv)
    
    region = robust_inference(*res, X, Z, D, y, grid=np.linspace(0, 20000, 10000))
    beta = np.mean(res[7] * res[2]) / np.mean(res[6] * res[2])
    var_beta = np.mean((res[6] - beta * res[7])**2 * res[2]**2) / np.mean(res[2]**2)**2
    se_beta = np.sqrt(var_beta / res[2].shape[0])
    t_stat = np.abs(beta / se_beta)
    
    print(f"Robust PLIV region: {np.min(region)} to {np.max(region)}")
    print(f"PLIV t-statistic (approx.): {t_stat:.3f}\n")
    
    return table_pliv


# Functions for IIV Analysis (similar try/except for FLAML part)

def iiv(X, Z, D, y, modely0, modely1, modeld0, modeld1, modelz, *, nfolds, trimming=0.01):
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=123)
    yhat0, yhat1 = np.zeros(y.shape), np.zeros(y.shape)
    Dhat0, Dhat1 = np.zeros(D.shape), np.zeros(D.shape)
    for train, test in cv.split(X, y):
        yhat0[test] = clone(modely0).fit(X.iloc[train][Z[train] == 0], y[train][Z[train] == 0]).predict(X.iloc[test])
        yhat1[test] = clone(modely1).fit(X.iloc[train][Z[train] == 1], y[train][Z[train] == 1]).predict(X.iloc[test])
        if np.mean(D[train][Z[train] == 0]) > 0:
            modeld0_ = clone(modeld0).fit(X.iloc[train][Z[train] == 0], D[train][Z[train] == 0])
            Dhat0[test] = modeld0_.predict_proba(X.iloc[test])[:, 1]
        if np.mean(D[train][Z[train] == 1]) < 1:
            modeld1_ = clone(modeld1).fit(X.iloc[train][Z[train] == 1], D[train][Z[train] == 1])
            Dhat1[test] = modeld1_.predict_proba(X.iloc[test])[:, 1]
        else:
            Dhat1[test] = 1
    yhat = yhat0 * (1 - Z) + yhat1 * Z
    Dhat = Dhat0 * (1 - Z) + Dhat1 * Z
    Zhat = cross_val_predict(modelz, X, Z, cv=cv, method='predict_proba', n_jobs=-1)[:, 1]
    Zhat = np.clip(Zhat, trimming, 1 - trimming)
    HZ = Z / Zhat - (1 - Z) / (1 - Zhat)
    drZ = yhat1 - yhat0 + (y - yhat) * HZ
    drD = Dhat1 - Dhat0 + (D - Dhat) * HZ
    point = np.mean(drZ) / np.mean(drD)
    psi = drZ - point * drD
    Jhat = np.mean(drD)
    var = np.mean(psi**2) / Jhat**2
    stderr = np.sqrt(var / X.shape[0])
    return point, stderr, yhat, Dhat, Zhat, y - yhat, D - Dhat, Z - Zhat, drZ, drD

def summary_iiv(point, stderr, yhat, Dhat, Zhat, resy, resD, resZ, drZ, drD, X, Z, D, y, *, name):
    return pd.DataFrame({
        'estimate': point,
        'stderr': stderr,
        'lower': point - 1.96 * stderr,
        'upper': point + 1.96 * stderr,
        'rmse y': np.sqrt(np.mean(resy**2)),
        'rmse D': np.sqrt(np.mean(resD**2)),
        'rmse Z': np.sqrt(np.mean(resZ**2)),
        'accuracy D': np.mean(np.abs(resD) < 0.5),
        'accuracy Z': np.mean(np.abs(resZ) < 0.5)
    }, index=[name])

def iivm_robust_inference(point, stderr, yhat, Dhat, Zhat, resy, resD, resZ, drZ, drD, X, Z, D, y, *, grid, alpha=0.05):
    n = X.shape[0]
    thr = scipy.stats.chi2.ppf(1 - alpha, df=1)
    accept = []
    for theta in grid:
        moment = drZ - theta * drD
        test = n * np.mean(moment)**2 / np.var(moment)
        if test <= thr:
            accept.append(theta)
    return accept

def run_iiv_analysis(data, desc, transformer):
    print(f"---------- IIV Analysis for {desc} ----------")
    y = data['net_tfa'].values
    Z = data['e401'].values
    D = data['p401'].values
    X = data.drop(drop_cols, axis=1)
    
    results = []
    cv = KFold(n_splits=5, shuffle=True, random_state=123)
    
    # Method 1: Lasso + Logistic for IIV
    lassoy = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv))
    lgrd = make_pipeline(transformer, StandardScaler(), LogisticRegressionCV(cv=cv))
    lgrz = make_pipeline(transformer, StandardScaler(), LogisticRegressionCV(cv=cv))
    res = iiv(X, Z, D, y, lassoy, lassoy, lgrd, lgrd, lgrz, nfolds=3)
    results.append(summary_iiv(*res, X, Z, D, y, name='lasso/logistic'))
    
    # Method 2: Random Forests for IIV
    rfy = make_pipeline(transformer, RandomForestRegressor(n_estimators=100, min_samples_leaf=10, ccp_alpha=0.001))
    rfd = make_pipeline(transformer, RandomForestClassifier(n_estimators=100, min_samples_leaf=10, ccp_alpha=0.001))
    rfz = make_pipeline(transformer, RandomForestClassifier(n_estimators=100, min_samples_leaf=10, ccp_alpha=0.001))
    res = iiv(X, Z, D, y, rfy, rfy, rfd, rfd, rfz, nfolds=3)
    results.append(summary_iiv(*res, X, Z, D, y, name='random forest'))
    
    # Method 3: Decision Trees for IIV
    dtry = make_pipeline(transformer, DecisionTreeRegressor(min_samples_leaf=10, ccp_alpha=0.001))
    dtrd = make_pipeline(transformer, DecisionTreeClassifier(min_samples_leaf=10, ccp_alpha=0.001))
    dtrz = make_pipeline(transformer, DecisionTreeClassifier(min_samples_leaf=10, ccp_alpha=0.001))
    res = iiv(X, Z, D, y, dtry, dtry, dtrd, dtrd, dtrz, nfolds=3)
    results.append(summary_iiv(*res, X, Z, D, y, name='decision trees'))
    
    # Method 4: Boosted Trees for IIV
    gbfy = make_pipeline(transformer, GradientBoostingRegressor(max_depth=2, n_iter_no_change=5))
    gbfd = make_pipeline(transformer, GradientBoostingClassifier(max_depth=2, n_iter_no_change=5))
    gbfz = make_pipeline(transformer, GradientBoostingClassifier(max_depth=2, n_iter_no_change=5))
    res = iiv(X, Z, D, y, gbfy, gbfy, gbfd, gbfd, gbfz, nfolds=3)
    results.append(summary_iiv(*res, X, Z, D, y, name='boosted trees'))
    
    # Method 5: AutoML for IIV via FLAML (using try/except)
    try:
        from flaml import AutoML
        X_trans = transformer.fit_transform(X)
        automl_reg = AutoML(time_budget=60, task='regression', early_stop=5, max_iter=100,
                            eval_method='cv', n_splits=3, metric='r2', verbose=0)
        automl_reg.fit(X_trans, y)
        best_reg = automl_reg.best_estimator
        automl_clf = AutoML(time_budget=60, task='classification', early_stop=5, max_iter=100,
                            eval_method='cv', n_splits=3, metric='r2', verbose=0)
        automl_clf.fit(X_trans, D)
        best_d = automl_clf.best_estimator
        automl_clf.fit(X_trans, Z)
        best_z = automl_clf.best_estimator
        modely0 = TransformedEstimator(transformer, clone(best_reg))
        modely1 = TransformedEstimator(transformer, clone(best_reg))
        modeld0 = TransformedEstimator(transformer, clone(best_d))
        modeld1 = TransformedEstimator(transformer, clone(best_d))
        modelz_auto = TransformedEstimator(transformer, clone(best_z))
        res_auto = iiv(X, Z, D, y, modely0, modely1, modeld0, modeld1, modelz_auto, nfolds=3)
        results.append(summary_iiv(*res_auto, X, Z, D, y, name='automl (semi-cfit)'))
    except Exception as e:
        print("FLAML AutoML for IIV failed, skipping AutoML method. Error:", e)
    
    table_iiv = pd.concat(results)
    print(table_iiv)
    
    region = iivm_robust_inference(*res, X, Z, D, y, grid=np.linspace(0, 20000, 10000))
    print(f"Robust IIV region: {np.min(region)} to {np.max(region)}\n")
    
    return table_iiv




In [11]:
print("Running analysis for Low Income (Bottom 25%) group...")
pliv_low = run_pliv_analysis(data_bottom, "Low Income (Bottom 25%)", transformer)
iiv_low  = run_iiv_analysis(data_bottom, "Low Income (Bottom 25%)", transformer)

Running analysis for Low Income (Bottom 25%) group...
---------- PLIV Analysis for Low Income (Bottom 25%) ----------
FLAML AutoML failed, skipping AutoML method. Error: `best_iteration` is only defined when early stopping is used.
                   estimate       stderr        lower        upper  \
double lasso    5768.455193  1701.445143  2433.622713  9103.287673   
lasso/logistic  5800.166817  1633.319043  2598.861493  9001.472141   
random forest   6545.081883  1697.472956  3218.034889  9872.128876   
decision tree   5794.913718  1664.321666  2532.843253  9056.984184   
boosted forest  6455.054651  1692.226298  3138.291108  9771.818195   

                      rmse y    rmse D    rmse Z  accuracy D  accuracy Z  
double lasso    13446.444167  0.292899  0.342700    0.901130    0.847054  
lasso/logistic  13446.444167  0.298584  0.358910    0.901130    0.847054  
random forest   13482.012085  0.293862  0.344844    0.901130    0.846247  
decision tree   14288.351637  0.310102  0.36333

In [12]:
print("Running analysis for High Income (Top 25%) group...")
pliv_high = run_pliv_analysis(data_top, "High Income (Top 25%)", transformer)
iiv_high  = run_iiv_analysis(data_top, "High Income (Top 25%)", transformer)


Running analysis for High Income (Top 25%) group...
---------- PLIV Analysis for High Income (Top 25%) ----------
FLAML AutoML failed, skipping AutoML method. Error: `best_iteration` is only defined when early stopping is used.
                    estimate       stderr         lower         upper  \
double lasso    23577.436393  5048.340429  13682.689152  33472.183634   
lasso/logistic  23102.880506  5051.372643  13202.190124  33003.570887   
random forest   22610.619713  5220.651521  12378.142733  32843.096694   
decision tree   19271.997928  6855.090475   5836.020598  32707.975259   
boosted forest  24267.441005  5296.199299  13886.890379  34647.991631   

                       rmse y    rmse D    rmse Z  accuracy D  accuracy Z  
double lasso     91149.933462  0.487619  0.483549    0.599435    0.591367  
lasso/logistic   91149.933462  0.487782  0.482933    0.589351    0.600242  
random forest    94743.172143  0.491518  0.484511    0.574425    0.608713  
decision tree   100253.218084

For bottom 25%: We see high consistency in both PLV and IIV analysis, with one notable exception being decision tree under IIV, which output a value of ~1400 where others were clustered between 5000 and 6000.

For top 25%: We see almost the exact same consistency at this income level as well, with the same exception for decision trees under IIV, which output a negative value whereas the rest were clustered between 22000 and 24000. 

Based on these results, we can in fact conclude that there exists heterogeneity with respect to 401k effect on income levels. At lower incomes, it seems to have a lower average affect on income, whereas this effect is much higher at the higher income levels. 

## b)

The notebook only provides code for semi-crossfitting with automl. Using the pseudo code from the lecture notes, also implement semi-crossfitting with stacking (see also prior notebook on estimating effect of eligibility of 401k), where we pass a list of models for each input model parameter, then we fit and predict each of the models in a cross-fitting manner and using all the out of fold predictions we select a weighted combination based on OLS regression to combine all the predictions. We then use this weighted average of predictions as the nuisance predictions for each sample. After implementing this, report the same metrics as the metrics that the notebook reports when doing semi-crossfitting with automl and use the list of models as the ones that the notebook used unilaterally (e.g. for the outcome use a list of [lasso, random forest, gradient boosted forest] and for the treatment a list of [logistic regression, random forest classifier, gradient boosted classifier]). Do that for both the PLIV setting and the interactive IV setting.

In [7]:
import pandas as pd
import numpy as np
import scipy.stats
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.linear_model import LinearRegression, LogisticRegressionCV
from sklearn.ensemble import (RandomForestRegressor, RandomForestClassifier, 
                              GradientBoostingRegressor, GradientBoostingClassifier)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone
import warnings
warnings.simplefilter('ignore')
np.random.seed(1234)


# Data loading and variable definitions 

file = "https://raw.githubusercontent.com/CausalAIBook/MetricsMLNotebooks/main/data/401k.csv"
data_all = pd.read_csv(file)
data_all = data_all.dropna()

# Define drop columns (as in your code)
drop_cols = ['e401', 'p401', 'a401', 'tw', 'tfa', 'net_tfa', 'tfa_he',
             'hval', 'hmort', 'hequity',
             'nifa', 'net_nifa', 'net_n401', 'ira',
             'dum91', 'icat', 'ecat', 'zhat',
             'i1', 'i2', 'i3', 'i4', 'i5', 'i6', 'i7',
             'a1', 'a2', 'a3', 'a4', 'a5']

# Define outcome, treatment, instrument and controls.
y = data_all['net_tfa'].values
Z = data_all['e401'].values
D = data_all['p401'].values
# Controls: drop the specified columns.
X = data_all.drop(drop_cols, axis=1)


#transformer and helper wrapper

from formulaic import Formula
from sklearn.base import BaseEstimator, TransformerMixin

class FormulaTransformer(TransformerMixin, BaseEstimator):
    def __init__(self, formula, array=False):
        self.formula = formula
        self.array = array
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        df = Formula(self.formula).get_model_matrix(X)
        return df.values if self.array else df

transformer = FormulaTransformer(
    "0 + poly(age, degree=6, raw=True) + poly(inc, degree=8, raw=True) "
    "+ poly(educ, degree=4, raw=True) + poly(fsize, degree=2, raw=True) "
    "+ male + marr + twoearn + db + pira + hown", array=True)

# A simple wrapper to combine a transformer with an estimator.
class TransformedEstimator(BaseEstimator):
    def __init__(self, transformer, estimator):
        self.transformer = transformer
        self.estimator = estimator
    def fit(self, X, y):
        X_trans = self.transformer.fit_transform(X)
        self.estimator.fit(X_trans, y)
        return self
    def predict(self, X):
        X_trans = self.transformer.transform(X)
        return self.estimator.predict(X_trans)
    def predict_proba(self, X):
        X_trans = self.transformer.transform(X)
        return self.estimator.predict_proba(X_trans)


# Summary and Inference functions

def summary(point, stderr, yhat, Dhat, Zhat, resy, resD, resZ, epsilon, X, Z, D, y, *, name):
    return pd.DataFrame({
        'estimate': point,
        'stderr': stderr,
        'lower': point - 1.96 * stderr,
        'upper': point + 1.96 * stderr,
        'rmse y': np.sqrt(np.mean(resy**2)),
        'rmse D': np.sqrt(np.mean(resD**2)),
        'rmse Z': np.sqrt(np.mean(resZ**2)),
        'accuracy D': np.mean(np.abs(resD) < 0.5),
        'accuracy Z': np.mean(np.abs(resZ) < 0.5)
    }, index=[name])

def summary_iiv(point, stderr, yhat, Dhat, Zhat, resy, resD, resZ, drZ, drD, X, Z, D, y, *, name):
    return pd.DataFrame({
        'estimate': point,
        'stderr': stderr,
        'lower': point - 1.96 * stderr,
        'upper': point + 1.96 * stderr,
        'rmse y': np.sqrt(np.mean(resy**2)),
        'rmse D': np.sqrt(np.mean(resD**2)),
        'rmse Z': np.sqrt(np.mean(resZ**2)),
        'accuracy D': np.mean(np.abs(resD) < 0.5),
        'accuracy Z': np.mean(np.abs(resZ) < 0.5)
    }, index=[name])

def robust_inference(point, stderr, yhat, Dhat, Zhat, resy, resD, resZ, epsilon, X, Z, D, y, *, grid, alpha=0.05):
    n = X.shape[0]
    thr = scipy.stats.chi2.ppf(1 - alpha, df=1)
    accept = []
    for theta in grid:
        moment = (resy - theta * resD) * resZ
        test = n * np.mean(moment)**2 / np.var(moment)
        if test <= thr:
            accept.append(theta)
    return accept

def iivm_robust_inference(point, stderr, yhat, Dhat, Zhat, resy, resD, resZ, drZ, drD, X, Z, D, y, *, grid, alpha=0.05):
    n = X.shape[0]
    thr = scipy.stats.chi2.ppf(1 - alpha, df=1)
    accept = []
    for theta in grid:
        moment = drZ - theta * drD
        test = n * np.mean(moment)**2 / np.var(moment)
        if test <= thr:
            accept.append(theta)
    return accept


# Helper: Out‐of‐fold stacking prediction

def stack_cv_prediction(model_list, X, y, cv, is_classifier=False):
    """
    Compute out‐of‐fold predictions for each model in model_list and stack them via OLS.
    """
    n_samples = X.shape[0]
    n_models = len(model_list)
    base_preds = np.zeros((n_samples, n_models))
    
    for j, model in enumerate(model_list):
        if is_classifier and hasattr(model, "predict_proba"):
            preds = cross_val_predict(model, X, y, cv=cv, method='predict_proba', n_jobs=-1)[:, 1]
        else:
            preds = cross_val_predict(model, X, y, cv=cv, n_jobs=-1)
        base_preds[:, j] = preds

    # Use OLS (LinearRegression) to stack the base predictions.
    stacker = LinearRegression().fit(base_preds, y)
    final_preds = stacker.predict(base_preds)
    return final_preds, base_preds, stacker


# 1) PLIV stacking function

def dml_pliv_stacking(
    X, Z, D, y,
    modely_list,    # list of models for outcome Y
    modeld_list,    # list of models for treatment D
    modelz_list,    # list of models for instrument Z
    nfolds=3,
    classifier_d=False,
    classifier_z=False,
    random_state=123
):
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=random_state)
    
    # Outcome stacking
    yhat, _, _ = stack_cv_prediction(modely_list, X, y, cv, is_classifier=False)
    
    # Treatment stacking
    Dhat, _, _ = stack_cv_prediction(modeld_list, X, D, cv, is_classifier=classifier_d)
    
    # Instrument stacking
    Zhat, _, _ = stack_cv_prediction(modelz_list, X, Z, cv, is_classifier=classifier_z)
    
    resy = y - yhat
    resD = D - Dhat
    resZ = Z - Zhat
    point = np.mean(resy * resZ) / np.mean(resD * resZ)
    epsilon = resy - point * resD
    var = np.mean(epsilon**2 * resZ**2) / (np.mean(resD * resZ)**2)
    stderr = np.sqrt(var / X.shape[0])
    
    return point, stderr, yhat, Dhat, Zhat, resy, resD, resZ, epsilon

# 2) IIV stacking function
def iiv_stacking(
    X, Z, D, y,
    modely0_list, modely1_list,   # models for outcome in Z=0 and Z=1 groups
    modeld0_list, modeld1_list,   # models for treatment in Z=0 and Z=1 groups
    modelz_list,                  # models for instrument Z
    trimming=0.01,
    nfolds=3,
    random_state=123
):
    n = X.shape[0]
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=random_state)
    
    # Initialize storage for out‐of‐fold predictions
    y0_preds = np.zeros((n, len(modely0_list)))
    y1_preds = np.zeros((n, len(modely1_list)))
    d0_preds = np.zeros((n, len(modeld0_list)))
    d1_preds = np.zeros((n, len(modeld1_list)))
    
    for train_idx, test_idx in cv.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train = y[train_idx]
        Z_train = Z[train_idx]
        D_train = D[train_idx]
        
        # Outcome models for Z=0
        idx_z0 = (Z_train == 0)
        if np.sum(idx_z0) > 0:
            for j, model in enumerate(modely0_list):
                mdl = clone(model).fit(X_train[idx_z0], y_train[idx_z0])
                y0_preds[test_idx, j] = mdl.predict(X_test)
                
        # Outcome models for Z=1
        idx_z1 = (Z_train == 1)
        if np.sum(idx_z1) > 0:
            for j, model in enumerate(modely1_list):
                mdl = clone(model).fit(X_train[idx_z1], y_train[idx_z1])
                y1_preds[test_idx, j] = mdl.predict(X_test)
                
        # Treatment models for Z=0
        D_train_z0 = D_train[Z_train == 0]
        if np.std(D_train_z0) < 1e-10:
            d0_preds[test_idx, :] = np.mean(D_train_z0)
        else:
            for j, model in enumerate(modeld0_list):
                mdl = clone(model).fit(X_train[Z_train==0], D_train_z0)
                if hasattr(mdl, "predict_proba"):
                    d0_preds[test_idx, j] = mdl.predict_proba(X_test)[:,1]
                else:
                    d0_preds[test_idx, j] = mdl.predict(X_test)
                    
        # Treatment models for Z=1
        D_train_z1 = D_train[Z_train == 1]
        if np.std(D_train_z1) < 1e-10:
            d1_preds[test_idx, :] = np.mean(D_train_z1)
        else:
            for j, model in enumerate(modeld1_list):
                mdl = clone(model).fit(X_train[Z_train==1], D_train_z1)
                if hasattr(mdl, "predict_proba"):
                    d1_preds[test_idx, j] = mdl.predict_proba(X_test)[:,1]
                else:
                    d1_preds[test_idx, j] = mdl.predict(X_test)
    
    # Stack predictions on each subgroup via OLS.
    idx_all = np.arange(n)
    idx_z0_all = (Z == 0)
    idx_z1_all = (Z == 1)
    
    # Stack outcomes
    stacker_y0 = LinearRegression().fit(y0_preds[idx_z0_all], y[idx_z0_all])
    final_y0 = stacker_y0.predict(y0_preds)
    stacker_y1 = LinearRegression().fit(y1_preds[idx_z1_all], y[idx_z1_all])
    final_y1 = stacker_y1.predict(y1_preds)
    
    # Stack treatments
    if np.var(D[idx_z0_all]) > 1e-10:
        stacker_d0 = LinearRegression().fit(d0_preds[idx_z0_all], D[idx_z0_all])
        final_d0 = stacker_d0.predict(d0_preds)
    else:
        final_d0 = np.full(n, np.mean(D[idx_z0_all]))
    if np.var(D[idx_z1_all]) > 1e-10:
        stacker_d1 = LinearRegression().fit(d1_preds[idx_z1_all], D[idx_z1_all])
        final_d1 = stacker_d1.predict(d1_preds)
    else:
        final_d1 = np.full(n, np.mean(D[idx_z1_all]))
        
    # Combine according to observed Z.
    yhat = final_y0 * (1 - Z) + final_y1 * Z
    Dhat = final_d0 * (1 - Z) + final_d1 * Z
    
    # Instrument stacking via standard CV stacking.
    cv_z = KFold(n_splits=nfolds, shuffle=True, random_state=random_state)
    z_base_preds = np.zeros((n, len(modelz_list)))
    for j, model in enumerate(modelz_list):
        if set(np.unique(Z)) == {0,1} and hasattr(model, "predict_proba"):
            preds = cross_val_predict(model, X, Z, cv=cv_z, method='predict_proba', n_jobs=-1)[:,1]
        else:
            preds = cross_val_predict(model, X, Z, cv=cv_z, n_jobs=-1)
        z_base_preds[:, j] = preds
    stacker_z = LinearRegression().fit(z_base_preds, Z)
    Zhat = stacker_z.predict(z_base_preds)
    Zhat = np.clip(Zhat, trimming, 1 - trimming)
    
    # Doubly robust transformation for IIV.
    HZ = (Z / Zhat) - ((1 - Z) / (1 - Zhat))
    drZ = final_y1 - final_y0 + (y - yhat) * HZ
    drD = final_d1 - final_d0 + (D - Dhat) * HZ
    point = np.mean(drZ) / np.mean(drD)
    psi = drZ - point * drD
    Jhat = np.mean(drD)
    var = np.mean(psi**2) / (Jhat**2)
    stderr = np.sqrt(var / n)
    
    resy = y - yhat
    resD = D - Dhat
    resZ = Z - Zhat
    
    return point, stderr, yhat, Dhat, Zhat, resy, resD, resZ, drZ, drD


# Fallback functions (use only the first model from each list)

def dml_pliv_fallback(X, Z, D, y, modely, modeld, modelz, nfolds=3, classifier_d=False, classifier_z=False, random_state=123):
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=random_state)
    yhat = cross_val_predict(modely, X, y, cv=cv, n_jobs=-1)
    if classifier_d and hasattr(modeld, "predict_proba"):
        Dhat = cross_val_predict(modeld, X, D, cv=cv, method='predict_proba', n_jobs=-1)[:,1]
    else:
        Dhat = cross_val_predict(modeld, X, D, cv=cv, n_jobs=-1)
    if classifier_z and hasattr(modelz, "predict_proba"):
        Zhat = cross_val_predict(modelz, X, Z, cv=cv, method='predict_proba', n_jobs=-1)[:,1]
    else:
        Zhat = cross_val_predict(modelz, X, Z, cv=cv, n_jobs=-1)
    resy = y - yhat
    resD = D - Dhat
    resZ = Z - Zhat
    point = np.mean(resy * resZ) / np.mean(resD * resZ)
    epsilon = resy - point * resD
    var = np.mean(epsilon**2 * resZ**2) / (np.mean(resD * resZ)**2)
    stderr = np.sqrt(var / X.shape[0])
    return point, stderr, yhat, Dhat, Zhat, resy, resD, resZ, epsilon

def iiv_fallback(X, Z, D, y, modely0, modely1, modeld0, modeld1, modelz, nfolds=3, trimming=0.01, random_state=123):
    n = X.shape[0]
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=random_state)
    y0_preds = np.zeros(n)
    y1_preds = np.zeros(n)
    d0_preds = np.zeros(n)
    d1_preds = np.zeros(n)
    for train_idx, test_idx in cv.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train = y[train_idx]
        Z_train = Z[train_idx]
        D_train = D[train_idx]
        if np.any(Z_train==0):
            mdl_y0 = clone(modely0).fit(X_train[Z_train==0], y_train[Z_train==0])
            y0_preds[test_idx] = mdl_y0.predict(X_test)
            mdl_d0 = clone(modeld0).fit(X_train[Z_train==0], D_train[Z_train==0])
            if hasattr(mdl_d0, "predict_proba"):
                d0_preds[test_idx] = mdl_d0.predict_proba(X_test)[:,1]
            else:
                d0_preds[test_idx] = mdl_d0.predict(X_test)
        if np.any(Z_train==1):
            mdl_y1 = clone(modely1).fit(X_train[Z_train==1], y_train[Z_train==1])
            y1_preds[test_idx] = mdl_y1.predict(X_test)
            mdl_d1 = clone(modeld1).fit(X_train[Z_train==1], D_train[Z_train==1])
            if hasattr(mdl_d1, "predict_proba"):
                d1_preds[test_idx] = mdl_d1.predict_proba(X_test)[:,1]
            else:
                d1_preds[test_idx] = mdl_d1.predict(X_test)
    yhat = y0_preds * (1 - Z) + y1_preds * Z
    Dhat = d0_preds * (1 - Z) + d1_preds * Z
    cv2 = KFold(n_splits=nfolds, shuffle=True, random_state=random_state)
    if set(np.unique(Z)) == {0,1} and hasattr(modelz, "predict_proba"):
        Zhat = cross_val_predict(modelz, X, Z, cv=cv2, method='predict_proba', n_jobs=-1)[:,1]
    else:
        Zhat = cross_val_predict(modelz, X, Z, cv=cv2, n_jobs=-1)
    Zhat = np.clip(Zhat, trimming, 1-trimming)
    resy = y - yhat
    resD = D - Dhat
    resZ = Z - Zhat
    HZ = (Z / Zhat) - ((1-Z)/(1-Zhat))
    drZ = y1_preds - y0_preds + (y - yhat) * HZ
    drD = d1_preds - d0_preds + (D - Dhat) * HZ
    point = np.mean(drZ) / np.mean(drD)
    psi = drZ - point * drD
    Jhat = np.mean(drD)
    var = np.mean(psi**2) / (Jhat**2)
    stderr = np.sqrt(var / n)
    return point, stderr, yhat, Dhat, Zhat, resy, resD, resZ, drZ, drD


# Define base learners 

# Outcome models
lasso_y = make_pipeline(transformer, StandardScaler(), LassoCV(cv=5))
rf_y    = make_pipeline(transformer, RandomForestRegressor(n_estimators=100, min_samples_leaf=10, ccp_alpha=0.001))
gbf_y   = make_pipeline(transformer, GradientBoostingRegressor(max_depth=2, n_iter_no_change=5))
modely_list = [lasso_y, rf_y, gbf_y]

# Treatment models (binary, so use classifiers)
lgr_d = make_pipeline(transformer, StandardScaler(), LogisticRegressionCV(cv=5))
rf_d  = make_pipeline(transformer, RandomForestClassifier(n_estimators=100, min_samples_leaf=10, ccp_alpha=0.001))
gbf_d = make_pipeline(transformer, GradientBoostingClassifier(max_depth=2, n_iter_no_change=5))
modeld_list = [lgr_d, rf_d, gbf_d]

# Instrument models (binary)
lgr_z = make_pipeline(transformer, StandardScaler(), LogisticRegressionCV(cv=5))
rf_z  = make_pipeline(transformer, RandomForestClassifier(n_estimators=100, min_samples_leaf=10, ccp_alpha=0.001))
gbf_z = make_pipeline(transformer, GradientBoostingClassifier(max_depth=2, n_iter_no_change=5))
modelz_list = [lgr_z, rf_z, gbf_z]

# For IIV, we use clones for Z=0 and Z=1 separately.
modely0_list = [clone(m) for m in modely_list]
modely1_list = [clone(m) for m in modely_list]
modeld0_list = [clone(m) for m in modeld_list]
modeld1_list = [clone(m) for m in modeld_list]
modelz_iiv_list = [clone(m) for m in modelz_list]


# Main Execution with try/except fallback for PLIV and IIV stacking analyses

grid = np.linspace(0, 20000, 10000)

print("---------- Running PLIV Stacking Analysis ----------")
try:
    pliv_res = dml_pliv_stacking(X, Z, D, y,
                                 modely_list=modely_list,
                                 modeld_list=modeld_list,
                                 modelz_list=modelz_list,
                                 nfolds=3,
                                 classifier_d=True,
                                 classifier_z=True)
    table_stack = summary(*pliv_res, X, Z, D, y, name='stacking (PLIV)')
    print(table_stack)
    
    region = robust_inference(*pliv_res, X, Z, D, y, grid=grid)
    print(f"Robust PLIV region: {np.min(region)} to {np.max(region)}")
except Exception as e:
    print("Error in PLIV stacking:", e)
    print("Running fallback PLIV using first models only.")
    pliv_res = dml_pliv_fallback(X, Z, D, y,
                                 modely=modely_list[0],
                                 modeld=modeld_list[0],
                                 modelz=modelz_list[0],
                                 nfolds=3,
                                 classifier_d=True,
                                 classifier_z=True)
    table_stack = summary(*pliv_res, X, Z, D, y, name='fallback (PLIV)')
    print(table_stack)
    region = robust_inference(*pliv_res, X, Z, D, y, grid=grid)
    print(f"Robust PLIV region (fallback): {np.min(region)} to {np.max(region)}")
    
print("\n---------- Running IIV Stacking Analysis ----------")
try:
    iiv_res = iiv_stacking(X, Z, D, y,
                           modely0_list, modely1_list,
                           modeld0_list, modeld1_list,
                           modelz_iiv_list,
                           trimming=0.01,
                           nfolds=3)
    table_iiv = summary_iiv(*iiv_res, X, Z, D, y, name='stacking (IIV)')
    print(table_iiv)
    
    region_iiv = iivm_robust_inference(*iiv_res, X, Z, D, y, grid=grid)
    print(f"Robust IIV region: {np.min(region_iiv)} to {np.max(region_iiv)}")
except Exception as e:
    print("Error in IIV stacking:", e)
    print("Running fallback IIV using first models only.")
    iiv_res = iiv_fallback(X, Z, D, y,
                           modely0=modely0_list[0],
                           modely1=modely1_list[0],
                           modeld0=modeld0_list[0],
                           modeld1=modeld1_list[0],
                           modelz=modelz_iiv_list[0],
                           nfolds=3,
                           trimming=0.01)
    table_iiv = summary_iiv(*iiv_res, X, Z, D, y, name='fallback (IIV)')
    print(table_iiv)
    region_iiv = iivm_robust_inference(*iiv_res, X, Z, D, y, grid=grid)
    print(f"Robust IIV region (fallback): {np.min(region_iiv)} to {np.max(region_iiv)}")


---------- Running PLIV Stacking Analysis ----------
                    estimate      stderr        lower         upper  \
stacking (PLIV)  12925.98142  1875.43089  9250.136875  16601.825965   

                       rmse y    rmse D    rmse Z  accuracy D  accuracy Z  
stacking (PLIV)  53925.153227  0.414402  0.443224    0.745134    0.690066  
Robust PLIV region: 9250.925092509251 to 16603.6603660366

---------- Running IIV Stacking Analysis ----------
                    estimate       stderr        lower         upper  \
stacking (IIV)  11180.221588  1637.570421  7970.583563  14389.859614   

                      rmse y    rmse D    rmse Z  accuracy D  accuracy Z  
stacking (IIV)  53698.051422  0.273537  0.443065    0.890973    0.692688  
Robust IIV region: 7970.79707970797 to 14391.439143914391


# Question 3

## a



We begin with the two moment conditions:
  
$$
\mathbb{E}\Big[\Big(\tilde{Y} - \alpha\,\tilde{D} - \delta'\,\tilde{S}\Big)\,\tilde{D}\Big] = 0,
$$

$$
\mathbb{E}\Big[\Big(\tilde{Y} - \alpha\,\tilde{D} - \delta'\,\tilde{S}\Big)\,\tilde{Q}\Big] = 0.
$$

These conditions can be compactly written as a vector of moment equations:

$$
\mathbb{E}\Big[m(W; \theta, \eta)\Big] = 0,
$$

where the parameter vector is

$$
\theta = (\alpha,\, \delta),
$$

and the moment function is defined as

$$
m(W; \theta, \eta) = \begin{pmatrix}
\Big(\tilde{Y} - \alpha\,\tilde{D} - \delta'\,\tilde{S}\Big)\,\tilde{D} \\
\Big(\tilde{Y} - \alpha\,\tilde{D} - \delta'\,\tilde{S}\Big)\,\tilde{Q}
\end{pmatrix}.
$$

Here, the residualized variables are obtained by “partialling out” the effect of controls $X$:
  
$$
\tilde{Y} = Y - \mathbb{E}[Y|X], \quad \tilde{D} = D - \mathbb{E}[D|X], \quad \tilde{S} = S - \mathbb{E}[S|X], \quad \tilde{Q} = Q - \mathbb{E}[Q|X].
$$

The collection of nuisance functions, denoted by $\eta$, includes the conditional expectation functions (used to form the residuals) and also the projection (or “best linear prediction”) step. In particular, when the technical instrument dimension exceeds that of the technical treatment, we estimate the projection matrix $B$ from regressing $\tilde{D}$ on $\tilde{Q}$:
  
$$
B = \mathbb{E}\Big[\tilde{V}\,\tilde{V}'\Big]^{-1} \mathbb{E}\Big[\tilde{V}\,\tilde{D}\Big],
$$

with $\tilde{V} = (\tilde{D},\, \tilde{Q})$. This projection is used to form the “new” instrument that enters our moment conditions.

---

### Framing as a Neyman-Orthogonal Moment Condition

The key idea is to express the estimation of $\theta = (\alpha, \delta)$ via the moment function $m(W; \theta, \eta)$ that depends on additional nuisance parameters $\eta$. Our estimator solves the empirical analogue

$$
\frac{1}{n}\sum_{i=1}^n m(W_i; \theta, \eta) = 0,
$$

with $\eta$ estimated separately (using, for example, cross-fitting).

**Neyman Orthogonality** requires that the moment function be locally insensitive to small errors in the estimation of $\eta$. Formally, if we perturb the nuisance functions by a small amount $h$, the Gateaux derivative at the true nuisance parameter $\eta_0$ should satisfy

$$
\frac{\partial}{\partial t}\Bigg|_{t=0} \mathbb{E}\Big[m(W; \theta_0, \eta_0 + t\, h)\Big] = 0.
$$

In our setup this orthogonality is achieved because:

1. **Residualization:**  
   The construction of $\tilde{Y}$, $\tilde{D}$, $\tilde{S}$, and $\tilde{Q}$ via regressing out the controls ensures that any estimation error in $\mathbb{E}[\,\cdot\,|X]$ is orthogonal to the residuals. In other words, the residuals are by construction uncorrelated with the nuisance estimates.

2. **Projection Step:**  
   The use of the projection matrix $B$ in forming the technical instruments further “purges” the variation that could be contaminated by errors in the nuisance estimates. Because $B$ is obtained by an OLS projection, its estimation error has a first‐order negligible effect on the moment conditions.

3. **Moment Structure:**  
   The moment function is linear in the residualized variables. Thus, any small error in the nuisance components (which enter additively when forming $\tilde{Y}$, $\tilde{D}$, etc.) does not change the first-order behavior of the moment condition. This guarantees that the derivative of the expectation of $m(W; \theta, \eta)$ with respect to $\eta$ is zero at $\eta_0$.

---

### Conclusion

By casting the estimation of $(\alpha, \delta)$ into the framework

$$
\mathbb{E}\Big[\begin{pmatrix}
\Big(\tilde{Y} - \alpha\,\tilde{D} - \delta'\,\tilde{S}\Big)\,\tilde{D} \\
\Big(\tilde{Y} - \alpha\,\tilde{D} - \delta'\,\tilde{S}\Big)\,\tilde{Q}
\end{pmatrix}\Big] = 0,
$$

we are in the standard setting of moment-based estimation with nuisance parameters. The residualization and projection steps ensure that this vector of moment equations satisfies the Neyman orthogonality property, making our estimator robust to first-order errors in the estimation of the nuisance functions.




## b

Let the parameter vector be 
$$
\theta = (\alpha,\, \delta),
$$ 
and consider the moment condition
$$
\mathbb{E}\Big[\Big(\tilde{Y} - \alpha\,\tilde{D} - \delta'\,\tilde{S}\Big)\,\tilde{Z}\Big] = 0,
$$
where the instrument vector is
$$
\tilde{Z} = (\tilde{D},\, \tilde{Q})
$$
and the regressor vector is
$$
\tilde{W} = (\tilde{D},\, \tilde{S}).
$$

Under the general theorem for parameters defined via Neyman orthogonal linear moment restrictions, the estimator $\hat{\theta}$ that solves the empirical moment equations is asymptotically normal:
$$
\sqrt{n}(\hat{\theta} - \theta_0) \overset{d}{\to} N(0, V),
$$
with asymptotic covariance matrix
$$
V = A^{-1} B (A^{-1})',
$$

where

$$
A = \mathbb{E}\left[\frac{\partial m(W; \theta, \eta)}{\partial \theta}\right],
$$ 

and 

$$
B = \mathbb{E}\Big[m(W; \theta, \eta)m(W; \theta, \eta)'\Big].
$$

A numerically equivalent estimation algorithm uses a two-stage least squares (2SLS) approach on the residualized system. The procedure is as follows:

1. **Residualization:**  
   For each observation $i$, estimate the nuisance functions using cross-fitting and obtain the residuals:
   $$
   \tilde{Y}_i = Y_i - \widehat{\mathbb{E}}[Y|X_i],\quad \tilde{D}_i = D_i - \widehat{\mathbb{E}}[D|X_i],
   $$
   $$
   \tilde{S}_i = S_i - \widehat{\mathbb{E}}[S|X_i],\quad \tilde{Q}_i = Q_i - \widehat{\mathbb{E}}[Q|X_i].
   $$

2. **First-Stage Projection:**  
   Regress $\tilde{D}$ on $\tilde{Q}$ via OLS (fitting the low-dimensional projection matrix on all the data) to obtain the fitted values
   $$
   \hat{D}_i = B\,\tilde{Q}_i.
   $$
   Form the instrument vector for the second stage as
   $$
   \hat{Z}_i = (\hat{D}_i,\, \tilde{Q}_i).
   $$

3. **Second-Stage Regression (2SLS):**  
   Run OLS of $\tilde{Y}$ on $\tilde{D}$ and $\tilde{S}$ using the instruments $\hat{Z}_i$ to obtain the estimator 
   $$
   \hat{\theta} = (\hat{\alpha},\, \hat{\delta}).
   $$

4. **Residual Calculation:**  
   Compute the second-stage residuals:
   $$
   \hat{u}_i = \tilde{Y}_i - \hat{\alpha}\,\tilde{D}_i - \hat{\delta}'\,\tilde{S}_i.
   $$

5. **Asymptotic Variance Estimation:**  
   The asymptotic covariance matrix for $\hat{\theta}$ is estimated by the standard 2SLS formula:
   $$
   \widehat{V} = \left(\frac{1}{n}\sum_{i=1}^n \hat{Z}_i\,\tilde{W}_i'\right)^{-1} \left(\frac{1}{n}\sum_{i=1}^n \hat{u}_i^2\, \hat{Z}_i\,\hat{Z}_i'\right) \left(\frac{1}{n}\sum_{i=1}^n \hat{Z}_i\,\tilde{W}_i'\right)^{-1\prime},
   $$
   where
   $$
   \tilde{W}_i = (\tilde{D}_i,\, \tilde{S}_i).
   $$

6. **Standard Error for $\hat{\alpha}$:**  
   The asymptotic variance of $\hat{\alpha}$ is the $(1,1)$ element of $\widehat{V}$:
   $$
   \text{Var}(\hat{\alpha}) \approx \widehat{V}_{11},
   $$
   so the standard error is
   $$
   \text{SE}(\hat{\alpha}) = \sqrt{\widehat{V}_{11}}.
   $$

**Algorithm Summary:**

- **Step 1:** Residualize $Y$, $D$, $S$, and $Q$ by regressing each on $X$ (using cross-fitting) to obtain $\tilde{Y}$, $\tilde{D}$, $\tilde{S}$, and $\tilde{Q}$.
- **Step 2:** Regress $\tilde{D}$ on $\tilde{Q}$ via OLS to obtain $B$ and fitted values $\hat{D}$; construct $\hat{Z} = (\hat{D}, \tilde{Q})$.
- **Step 3:** Run 2SLS: regress $\tilde{Y}$ on $(\tilde{D}, \tilde{S})$ using instruments $\hat{Z}$ to obtain $\hat{\theta} = (\hat{\alpha}, \hat{\delta})$.
- **Step 4:** Compute residuals $\hat{u}_i = \tilde{Y}_i - \hat{\alpha}\,\tilde{D}_i - \hat{\delta}'\,\tilde{S}_i$.
- **Step 5:** Estimate
  $$
  \widehat{V} = \left(\frac{1}{n}\sum_{i=1}^n \hat{Z}_i\,\tilde{W}_i'\right)^{-1} \left(\frac{1}{n}\sum_{i=1}^n \hat{u}_i^2\, \hat{Z}_i\,\hat{Z}_i'\right) \left(\frac{1}{n}\sum_{i=1}^n \hat{Z}_i\,\tilde{W}_i'\right)^{-1\prime}.
  $$
- **Step 6:** The standard error for $\hat{\alpha}$ is given by
  $$
  \text{SE}(\hat{\alpha}) = \sqrt{\widehat{V}_{11}}.
  $$
